In [8]:
import requests
import math
import random
import pandas as pd
from requests.adapters import HTTPAdapter, Retry
import time

#Use this to initialize the required sample size, specifically using the following metrics: 
# of total households in Atlanta, confidence level, and confidence interval
#https://www.surveysystem.com/sscalc.htm
#250,000 total housing units in Atlanta, GA from https://datacommons.org/place/geoId/1304000?category=Housing
# About 90,000 in the dataset total households in Realty Mole dataset

#Sample Size of 1600 gdetermined knowing that we want a dataset that represents the total population of households with 95% confidence and a C.I of +-2.5%
sample_size = 1600

#Total number of requests required, knowing that each pull returns a max of 50 households
pull_num = math.ceil(sample_size/50)
#Randomizes offset to randomize properties return within the dataset
offset = (random.randint(0,1600))*50
df = pd.DataFrame()

#GET Property Records that corresponds to Atlanta-Sandy Springs-Alpharetta, GA MSA using Latitude, Longitude, and Radius
while df.shape[0] < sample_size :
    
    s = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504, 204])
    s.mount('https://', HTTPAdapter(max_retries=retries))
    
    url = "https://realty-mole-property-api.p.rapidapi.com/properties"

    querystring = {"state":"GA", "City":"Atlanta", "offset":offset}

    headers = {
        "X-RapidAPI-Key": "caf41df26amshefe17a89e357781p1bc71bjsnf8eceed62dab",
        "X-RapidAPI-Host": "realty-mole-property-api.p.rapidapi.com"
    }
    response = s.request("GET", url, headers=headers, params=querystring)
    offset = (random.randint(0,1600))*50
    data = response.json()
    df_add = pd.DataFrame(data)
    #removes blank response values for lastSalePrice
    if 'lastSalePrice' in df_add.columns:
        df_add = df_add[df_add['lastSalePrice'].notna()]
        df = df.append(df_add)
    time.sleep(.5)

#outputs to csv file
df.to_csv("data\property_data.csv")

1604


In [9]:
df = pd.read_csv("data\property_data.csv")
display(df)
print(df.columns)

,Unnamed: 0,addressLine1,city,state,zipCode,formattedAddress,assessorID,bedrooms,county,legalDescription,...,features,lastSalePrice,lastSaleDate,owner,taxAssessment,propertyTaxes,id,longitude,latitude,addressLine2
0,0,2469 Benson Rdg,Lithonia,GA,30058,"2469 Benson Rdg, Lithonia, GA 30058",16-218 -03-170,3.0,DeKalb County,PROVIDENCE POINT S/D - UNIT 3-A BLOCK G LOT 74...,...,"{'cooling': True, 'coolingType': 'Central', 'e...",160000.0,2018-01-23T00:00:00.000Z,"{'names': ['Kayla Monique Dozier'], 'mailingAd...","{'2019': {'value': 69000, 'land': 15000, 'impr...","{'2019': {'total': 2322}, '2020': {'total': 23...","2469-Benson-Rdg,-Lithonia,-GA-30058",-84.067653,33.723977,NaN
1,2,2416 Rambling Way,Lithonia,GA,30058,"2416 Rambling Way, Lithonia, GA 30058",16-103 -12-007,3.0,Dekalb,STONEBRIDGE CREEK ESTATES UNIT ONE BLOCK E LOT...,...,"{'cooling': True, 'coolingType': 'Central', 'e...",105000.0,2017-06-09T00:00:00.000Z,"{'names': ['Bracha Sokolic'], 'mailingAddress'...","{'2019': {'value': 42000, 'land': 8000, 'impro...","{'2019': {'total': 2156}, '2021': {'total': 31...","2416-Rambling-Way,-Lithonia,-GA-30058",-84.130286,33.719805,NaN
2,3,2420 Briar Knoll Rd,Lithonia,GA,30058,"2420 Briar Knoll Rd, Lithonia, GA 30058",16-058 -01-076,3.0,Dekalb,COVINGTON PLACE BLOCK A LOT 36 1-3- 75 X 164 X...,...,"{'architectureType': 'Split Level', 'cooling':...",11569148.0,2017-05-15T00:00:00.000Z,"{'names': ['PFIN II F LLC'], 'mailingAddress':...","{'2019': {'value': 52080, 'land': 7200, 'impro...","{'2019': {'total': 2629}, '2021': {'total': 31...","2420-Briar-Knoll-Rd,-Lithonia,-GA-30058",-84.160979,33.726402,NaN
3,5,2219 Johnson Creek Dr,Lithonia,GA,30058,"2219 Johnson Creek Dr, Lithonia, GA 30058",16-219 -06-005,4.0,Dekalb,PROVIDENCE POINT S/D - UNIT 3-A BLOCK G LOT 1 ...,...,"{'cooling': True, 'coolingType': 'Central', 'e...",100500.0,2011-11-30T00:00:00.000Z,"{'names': ['CHARLIE MAE PERRY'], 'mailingAddre...","{'2019': {'value': 72440, 'land': 15000, 'impr...","{'2019': {'total': 2183}, '2021': {'total': 25...","2219-Johnson-Creek-Dr,-Lithonia,-GA-30058",-84.067283,33.728890,NaN
4,6,2270 Briar Knoll Rd,Lithonia,GA,30058,"2270 Briar Knoll Rd, Lithonia, GA 30058",16-058 -04-005,4.0,Dekalb,COVINGTON PLACE BLOCK B LOT 93 1-3- 24 X 228 X...,...,"{'cooling': True, 'coolingType': 'Central', 'e...",123700.0,1997-06-04T00:00:00.000Z,"{'names': ['JACQUELIN DANIELS'], 'mailingAddre...","{'2019': {'value': 58800, 'land': 7200, 'impro...","{'2019': {'total': 2025}, '2020': {'total': 21...","2270-Briar-Knoll-Rd,-Lithonia,-GA-30058",-84.160283,33.722459,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,24,2300 Greenway Dr,Decatur,GA,30035,"2300 Greenway Dr, Decatur, GA 30035",15-131 -01-115,3.0,Dekalb,NaN,...,"{'architectureType': 'Split Level', 'cooling':...",90500.0,2018-08-29T00:00:00.000Z,"{'names': ['AMERICAN EQUITY FINANCIAL LLC'], '...","{'2021': {'value': 68000, 'land': 6000, 'impro...",{'2021': {'total': 3330}},"2300-Greenway-Dr,-Decatur,-GA-30035",-84.219227,33.721101,NaN
1600,33,2246 Clanton Ter,Decatur,GA,30034,"2246 Clanton Ter, Decatur, GA 30034",15-118 -10-021,3.0,DeKalb County,NaN,...,"{'architectureType': 'Ranch / Rambler', 'cooli...",35000.0,2014-03-14T00:00:00.000Z,"{'names': ['SFR ATL OWNER 3 LP'], 'mailingAddr...","{'2020': {'value': 43817, 'land': 7735, 'impro...",{'2020': {'total': 2266}},"2246-Clanton-Ter,-Decatur,-GA-30034",-84.298741,33.712506,NaN
1601,42,130 Village Park Dr,Newnan,GA,30265,"130 Village Park Dr, Newnan, GA 30265",SG9 732,4.0,Coweta,HSE/ LT A52 VILLAGE WALK PH 3 @ STILLWOOD,...,"{'cooling': True, 'coolingType': 'Central', 'f...",239900.0,2018-07-18T00:00:00.000Z,"{'names': ['CERBERUS SFR HOLDINGS II LP'], 'ma...","{'2020': {'value': 110904, 'land': 16000, 'imp...","{'2020': {'total': 3048}, '2021': {'total': 25...","130-Village-Park-Dr,-Newnan,-GA-30265",-84.747476,33.366311,NaN
1602,43,117 Denham Sq,Peachtree City,GA,30269,"117 Denham Sq, Peachtree City, GA 30269",0

Index(['Unnamed: 0', 'addressLine1', 'city', 'state', 'zipCode',
       'formattedAddress', 'assessorID', 'bedrooms', 'county',
       'legalDescription', 'ownerOccupied', 'squareFootage', 'subdivision',
       'yearBuilt', 'bathrooms', 'lotSize', 'propertyType', 'features',
       'lastSalePrice', 'lastSaleDate', 'owner', 'taxAssessment',
       'propertyTaxes', 'id', 'longitude', 'latitude', 'addressLine2'],
      dtype='object')
